In [1]:
import torch.utils.data as data

import random
import numbers
from PIL import Image, ImageMath
import os
import os.path
import numpy as np
import struct
import math

import torch
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import faiss
import time

import pcl
from PIL import Image, ImageDraw
import faiss

import timeit

%matplotlib qt5

In [2]:
class KNNBuilder_GPU:
    def __init__(self, k):
        self.k = k
        self.dimension = 3
        
        # we need only a StandardGpuResources per GPU
        self.res = faiss.StandardGpuResources()
        self.res.setTempMemoryFraction(0.1)
        self.flat_config = faiss.GpuIndexFlatConfig()
        self.flat_config.device = 0
        
    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.GpuIndexFlatL2(self.res, self.dimension, self.flat_config)  # dimension is 3
        index.add(database)
        return index
    
    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I
    
    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I
    

class KNNBuilder:
    def __init__(self, k):
        self.k = k
        self.dimension = 3

    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.IndexFlatL2(self.dimension)  # dimension is 3
        index.add(database)
        return index

    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I

    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I

In [3]:
def read_velodyne_bin(path):
    '''
    :param path:
    :return: homography matrix of the point cloud, N*4
    '''
    pc_list = []
    with open(path, 'rb') as f:
        content = f.read()
        pc_iter = struct.iter_unpack('ffff', content)
        for idx, point in enumerate(pc_iter):
            pc_list.append([point[0], point[1], point[2], point[3]])
    return np.asarray(pc_list, dtype=np.float32)


def read_calib(path):
    '''
    :param path: string pointed to the calib txt
    :return: P0(3x4), P2(3x4), Tr(3x4)
    '''
    P0 = None
    P2 = None
    Tr = None
    
    f = open(path, 'r')
    lines = [l.rstrip() for l in f.readlines()]
    f.close()
    for line in lines:
        line_splitted = line.split()
        if len(line_splitted) < 1:
            pass
        elif line_splitted[0] == 'P0:':
            P0 = np.asarray(line_splitted[1:], dtype=np.float64).reshape(3,4)
        elif line_splitted[0] == 'P2:':
            P2 = np.asarray(line_splitted[1:], dtype=np.float64).reshape(3,4)
        elif line_splitted[0] == 'R0_rect:':
            R0_rect = np.asarray(line_splitted[1:], dtype=np.float64).reshape(3,3)
        elif line_splitted[0] == 'Tr_velo_to_cam:' or line_splitted[0] == 'Tr:':
            Tr = np.asarray(line_splitted[1:], dtype=np.float64).reshape(3,4)
    
    assert P0 is not None
    assert P2 is not None
    assert Tr is not None
    
    return (P0, P2, Tr)


class PCSampler:
    def __init__(self, leaf_size, minimum_pc_num):       
        self.leaf_size = leaf_size
        self.minimum_pc_num = minimum_pc_num
    
    def sample_pc(self, pc, leaf_size):
        '''
        :param pc: input numpy array of Nx3
        :return: sampled_pc of Mx3
        '''
        cloud = pcl.PointCloud(pc)
        sor = cloud.make_voxel_grid_filter()
        sor.set_leaf_size(leaf_size, leaf_size, leaf_size)
        cloud_filtered = sor.filter()
        sampled_pc = np.asarray(cloud_filtered)
        
        return sampled_pc
    
    def sample_pc_wrapper(self, pc):
        '''
        ensure that the sampled pc is more than a certain amount
        '''
        retry_counter = 0
        
        sampled_pc = self.sample_pc(pc, self.leaf_size)
        while sampled_pc.shape[0] < self.minimum_pc_num:
            retry_counter += 1
            leaf_size = self.leaf_size - 0.04*retry_counter
            if leaf_size <= 0:
                break
            sampled_pc = self.sample_pc(pc, leaf_size)
        
        return sampled_pc
    

def transform_velo_to_camera(pc, Tr):
    # pc: Nx4, x,y,z,reflectance
    # Tr: 4x3
    assert pc.shape[1] == 4
    
    pc_coord = pc[:, 0:3]  # Nx3
    pc_coord_homo = np.concatenate((pc_coord, np.ones((pc_coord.shape[0], 1))), axis=1)  # Nx4
    pc_coord_homo_transposed = np.transpose(pc_coord_homo)
    pc_coord_cam = np.transpose(np.dot(Tr, pc_coord_homo_transposed))  # 3xN -> Nx3
    
    pc_cam = np.concatenate((pc_coord_cam, pc[:, 3:]), axis=1)
    return pc_cam


def project_cam_pc_to_img(pc, P):
    # pc: Nx3, P: 3x4
    assert pc.shape[1] == 3
    
    pc_homo = np.concatenate((pc, np.ones((pc.shape[0], 1))), axis=1)  # Nx4
    pc_homo_transposed = np.transpose(pc_homo)  # 4xN
    pc_projected = np.transpose(np.dot(P, pc_homo_transposed))  # Nx3
    
    pixels = pc_projected[:, 0:2] / pc_projected[:, 2:]
    return pixels
    
    
def crop_pc_within_img(pc, pixels):
    # pc: Nx4, x, y, z, reflectance
    # pixels: Nx2, x, y
    assert pc.shape[1] == 4
    
    width, height = 1242, 375
    x_valid_idx = np.logical_and(pixels[:, 0]>=0, pixels[:, 0]<=width)
    y_valid_idx = np.logical_and(pixels[:, 1]>=0, pixels[:, 1]<=height)
    pixel_valid_idx = np.logical_and(x_valid_idx, y_valid_idx)
    range_valid_idx = pc[:, 2] > 0
    valid_idx = np.logical_and(pixel_valid_idx, range_valid_idx)
    
    valid_pc = pc[valid_idx, :]
    return valid_pc


def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [4]:
def sample_from_bin(input_path, output_path, calib_path, leaf_size=0.2, minimum_pc_num=6000, 
                    radius_cutoff=90, outlier_filter=True):
    pc_sampler = PCSampler(leaf_size, minimum_pc_num)
    total_minimum_pc_num = 100000
    
    sample_num = len(os.listdir(input_path))
    for i in range(sample_num):
        
        file = os.path.join(input_path, '%06d.bin'%i)
        pc = read_velodyne_bin(file)
        
        # convert to camera coordinate, i.e., P0 coordinate
        P0, P2, Tr = read_calib(calib_path)
        pc_cam0 = transform_velo_to_camera(pc, Tr)  # Nx4
        
        # cutoff radius
        valid_index = np.linalg.norm(pc_cam0[:, [0, 2]], axis=1) < radius_cutoff
        pc_cam0 = np.ascontiguousarray(pc_cam0[valid_index, :], dtype=np.float32)
        
        # ------------------ verify the functions -------------------------------------------
        # ------------------ verify the functions -------------------------------------------
        
        # down sample using pcl voxelgrid filter, 
        # "all the points present will be approximated (i.e., downsampled) with their centroid"
        sampled_pc_cam0 = pc_sampler.sample_pc_wrapper(pc_cam0[:, 0:3].astype(np.float32))
        
        # outlier filter
        if outlier_filter == True:
#             start_t = timeit.default_timer()
            fil = pcl.PointCloud(sampled_pc_cam0).make_statistical_outlier_filter()
            fil.set_mean_k(50)
            fil.set_std_dev_mul_thresh(2)
            sampled_pc_cam0 = np.asarray(fil.filter())
#             stop_t = timeit.default_timer()
#             print('outlier filter: %f' % (stop_t - start_t))
        
        # KNN to get the reflectance, and measure the error introduced by voxel filter
        knn = KNNBuilder_GPU(k=1)
        index = knn.build_nn_index(np.ascontiguousarray(pc_cam0[:, 0:3], dtype=np.float32))
        D, I = knn.search_nn(index, sampled_pc_cam0.astype(np.float32), k=1)
        
        reflectance = pc_cam0[:, 3:][I[:,0]]
        sampled_pc_cam0 = np.concatenate((sampled_pc_cam0, reflectance), axis=1)
        # print(sampled_pc_cam0)
        # print(pc_cam0[I[2,0]])
        
        # so far it seems the sampled_pc_cam0 is ready for saving
        if outlier_filter == False:
            if not os.path.isdir(os.path.join(output_path, 'np_%.2f_%d_r%d'%(leaf_size, minimum_pc_num, radius_cutoff))):
                os.makedirs(os.path.join(output_path, 'np_%.2f_%d_r%d'%(leaf_size, minimum_pc_num, radius_cutoff)))
            np.save(os.path.join(output_path, 'np_%.2f_%d_r%d'%(leaf_size, minimum_pc_num, radius_cutoff), '%06d.npy'%i), sampled_pc_cam0)
        else:
            if not os.path.isdir(os.path.join(output_path, 'np_%.2f_%d_r%d_filtered'%(leaf_size, minimum_pc_num, radius_cutoff))):
                os.makedirs(os.path.join(output_path, 'np_%.2f_%d_r%d_filtered'%(leaf_size, minimum_pc_num, radius_cutoff)))
            np.save(os.path.join(output_path, 'np_%.2f_%d_r%d_filtered'%(leaf_size, minimum_pc_num, radius_cutoff), '%06d.npy'%i), sampled_pc_cam0)
        
        # debug
#         if True:
#             print(pc.shape)
#             print(pc_cam0.shape)
#             print(sampled_pc_cam0.shape)
#             print('mean distance from sampled pt to original pt: %f'%np.mean(D))
#             print('max distance from sampled pt to original pt: %f'%np.amax(D))
#             print('distance std sigma from sampled pt to original pt: %f'%np.std(D))
#             fig = plt.figure()
#             ax = Axes3D(fig)
#             ax.scatter(sampled_pc_cam0[:,0].tolist(), sampled_pc_cam0[:,1].tolist(), sampled_pc_cam0[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
#             axisEqual3D(ax)

#             fig = plt.figure()
#             ax = Axes3D(fig)
#             ax.scatter(pc_cam0[:,0].tolist(), pc_cam0[:,1].tolist(), pc_cam0[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
#             axisEqual3D(ax)

#             plt.ion()
#             plt.show()
            
#             break
                    
        
        if sampled_pc_cam0.shape[0] < total_minimum_pc_num:
            total_minimum_pc_num = sampled_pc_cam0.shape[0]
        
        if i % 100 == 0:
            print('iteration: %d, total_minimum_pc_num: %d' % (i, total_minimum_pc_num))
        
        
    return total_minimum_pc_num


In [ ]:
minimum_pc_num = 100000
for seq in range(11):
    begin_t = time.time()
    pc_num = sample_from_bin('/ssd/dataset/odometry/data_odometry_velodyne/sequences/%02d/velodyne'%(seq),
                             '/ssd/dataset/odometry/data_odometry_velodyne/numpy/%02d'%(seq),
                             '/ssd/dataset/odometry/calib/%02d/calib.txt'%(seq),
                             leaf_size=0.2, minimum_pc_num=20480, radius_cutoff=50, outlier_filter=False)
    print(time.time()-begin_t)
    
    if pc_num < minimum_pc_num:
        minimum_pc_num = pc_num

    print('minimum_pc_num: %d' % minimum_pc_num)
    


In [ ]:
minimum_pc_num = 100000

seq = 10

begin_t = time.time()
pc_num = sample_from_bin('/ssd/dataset/odometry/data_odometry_velodyne/sequences/%02d/velodyne'%(seq),
                         '/ssd/dataset/odometry/data_odometry_velodyne/numpy/%02d'%(seq),
                         '/ssd/dataset/odometry/calib/%02d/calib.txt'%(seq),
                         leaf_size=0.2, minimum_pc_num=20480, radius_cutoff=90, outlier_filter=False)
print(time.time()-begin_t)

if pc_num < minimum_pc_num:
    minimum_pc_num = pc_num

print('minimum_pc_num: %d' % minimum_pc_num)

In [ ]:
# get surface normal
def Surface_normals(cloud):
    ne = cloud.make_NormalEstimation()
    tree = cloud.make_kdtree()
    ne.set_SearchMethod(tree)
#     ne.set_RadiusSearch(2)
    ne.set_KSearch(9)
    cloud_normals = ne.compute()
    return cloud_normals


input_path = '/ssd/dataset/odometry/data_odometry_velodyne/numpy'
input_folder = 'np_0.20_20480_r90'
output_folder = 'np_0.20_20480_r90_sn'
for seq in range(11):
    sample_num = len(os.listdir(os.path.join(input_path, '%02d'%seq, input_folder)))
    for i in range(sample_num):
        pc_np = np.load(os.path.join(input_path, '%02d'%seq, input_folder, '%06d.npy'%i))  # Nx4, x,y,z,ref
        cloud = pcl.PointCloud(pc_np[:, 0:3])
        
        sn = Surface_normals(cloud)
        sn_np = (np.asarray(sn.to_array()))  # Nx4, nx,ny,nz,curvature
        
        output_np = np.concatenate((pc_np[:, 0:3], sn_np, pc_np[:, 3:]), axis=1)
        
        if not os.path.isdir(os.path.join(input_path, '%02d'%seq, output_folder)):
            os.makedirs(os.path.join(input_path, '%02d'%seq, output_folder))
        np.save(os.path.join(input_path, '%02d'%seq, output_folder, '%06d.npy'%i), output_np)
        
        if i%100==0:
            print('seq %d - %d' % (seq, i))


In [ ]:
input_path = '/ssd/dataset/odometry/data_odometry_velodyne/numpy'
input_folder = 'np_0.20_20480_r90_filtered'
output_folder = 'np_0.20_20480_r90_filtered_sn'

pc_np = np.load(os.path.join(input_path, '06', output_folder, '000100.npy'))
print(pc_np.shape)

# mean of three axis
print('max of three axis')
print(np.max(pc_np[:, 0:3], axis=0))
print('min of three axis')
print(np.min(pc_np[:, 0:3], axis=0))

print(np.linalg.norm(pc_np[:, 3:6], axis=1))
# reflectance
print(np.mean(pc_np[:, 7]))
print(np.max(pc_np[:, 7]))
print(np.min(pc_np[:, 7]))


fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(pc_np[:,0].tolist(), pc_np[:,1].tolist(), pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
axisEqual3D(ax)

plt.ion()
plt.show()